In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import os
import glob

In [2]:
wcup = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Data\WorldCUP\WorldCupMatches.csv')

In [3]:
wcup.head()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA


In [105]:
wcup[(wcup['Home Team Name'] == 'Brazil') | (wcup['Away Team Name'] == 'Brazil')]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
11,1930.0,20 Jul 1930 - 13:00,Group 2,Estadio Centenario,Montevideo,Brazil,4.0,0.0,Bolivia,,25466.0,1.0,0.0,BALWAY Thomas (FRA),MATEUCCI Francisco (URU),VALLEJO Gaspar (MEX),201.0,1091.0,BRA,BOL
23,1934.0,27 May 1934 - 16:30,Preliminary round,Luigi Ferraris,Genoa,Spain,3.0,1.0,Brazil,,21000.0,3.0,0.0,BIRLEM Alfred (GER),CARMINATI Ettore (ITA),IVANCSICS Mihaly (HUN),204.0,1111.0,ESP,BRA
40,1938.0,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6.0,5.0,Poland,Brazil win after extra time,13452.0,0.0,0.0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206.0,1150.0,BRA,POL
44,1938.0,12 Jun 1938 - 17:00,Quarter-finals,Stade du Parc Lescure,Bordeaux,Brazil,1.0,1.0,Czechoslovakia,,22021.0,0.0,0.0,VON HERTZKA Pal (HUN),SCARPI Giuseppe (ITA),DE LA SALLE Charles (FRA),429.0,1152.0,BRA,TCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,2014.0,12 Jul 2014 - 17:00,Play-off for third place,Estadio Nacional,Brasilia,Brazil,0.0,3.0,Netherlands,,68034.0,0.0,2.0,HAIMOUDI Djamel (ALG),ACHIK Redouane (MAR),ETCHIALI Abdelhak (ALG),255957.0,300186502.0,BRA,NED
836,2014.0,28 Jun 2014 - 13:00,Round of 16,Estadio Mineirao,Belo Horizonte,Brazil,1.0,1.0,Chile,Brazil win on penalties (3 - 2),57714.0,0.0,0.0,WEBB Howard (ENG),MULLARKEY Michael (ENG),Darren CANN (ENG),255951.0,300186487.0,BRA,CHI
845,2014.0,04 Jul 2014 - 17:00,Quarter-finals,Estadio Castelao,Fortaleza,Brazil,2.0,1.0,Colombia,,60342.0,1.0,0.0,Carlos VELASCO CARBALLO (ESP),ALONSO FERNANDEZ Roberto (ESP),YUSTE Juan (ESP),255953.0,300186461.0,BRA,COL
848,2014.0,08 Jul 2014 - 17:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1.0,7.0,Germany,,58141.0,0.0,5.0,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955.0,300186474.0,BRA,GER


In [4]:
wcup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  852 non-null    float64
 1   Datetime              852 non-null    object 
 2   Stage                 852 non-null    object 
 3   Stadium               852 non-null    object 
 4   City                  852 non-null    object 
 5   Home Team Name        852 non-null    object 
 6   Home Team Goals       852 non-null    float64
 7   Away Team Goals       852 non-null    float64
 8   Away Team Name        852 non-null    object 
 9   Win conditions        852 non-null    object 
 10  Attendance            850 non-null    float64
 11  Half-time Home Goals  852 non-null    float64
 12  Half-time Away Goals  852 non-null    float64
 13  Referee               852 non-null    object 
 14  Assistant 1           852 non-null    object 
 15  Assistant 2          

In [5]:
wcup.describe()

,Year,Home Team Goals,Away Team Goals,Attendance,Half-time Home Goals,Half-time Away Goals,RoundID,MatchID
count,852.000000,852.000000,852.000000,850.000000,852.000000,852.000000,8.520000e+02,8.520000e+02
mean,1985.089202,1.811033,1.022300,45164.800000,0.708920,0.428404,1.066177e+07,6.134687e+07
std,22.448825,1.610255,1.087573,23485.249247,0.937414,0.691252,2.729613e+07,1.110572e+08
min,1930.000000,0.000000,0.000000,2000.000000,0.000000,0.000000,2.010000e+02,2.500000e+01
25%,1970.000000,1.000000,0.000000,30000.000000,0.000000,0.000000,2.620000e+02,1.188750e+03
50%,1990.000000,2.000000,1.000000,41579.500000,0.000000,0.000000,3.370000e+02,2.191000e+03
75%,2002.000000,3.000000,2.000000,61374.500000,1.000000,1.000000,2.497220e+05,4.395006e+07
max,2014.000000,10.000000,7.000000,173850.000000,6.000000,5.000000,9.741060e+07,3.001865e+08


In [6]:
ecu_opp = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Data\WorldCUP\archive\International_Football_History_Countries\Ecuador\Opponents_History.csv')

In [31]:
ecu_opp.head(3)

,Squad,MP,W,D,L,GF,GA,GD,PPM,Head-to-Head
0,bo Bolivia,16,13,2,1,41,15,+26,2.56,Head-to-Head
1,pe Peru,16,7,5,4,26,18,+8,1.63,Head-to-Head
2,ve Venezuela,15,7,4,4,23,16,+7,1.67,Head-to-Head


In [43]:
oppo_list = glob.glob(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Data\WorldCUP\archive\International_Football_History_Countries\*\Opponents_History.csv')

In [44]:
ecu_opp['Home_country'] = oppo_list[9].split('\\')[10]

In [82]:
opponent_history = pd.DataFrame()

for i,path in enumerate(oppo_list):
    team = pd.read_csv(path)
    team['Main'] = oppo_list[i].split('\\')[10]
    opponent_history = pd.concat([opponent_history, team], axis=0)

In [92]:
opp_history = opponent_history[opponent_history.Squad != 'Squad']

In [103]:
bra_wc = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Data\WorldCUP\archive\International_Football_History_Countries\Brazil\Worldcups_History.csv')

In [104]:
bra_wc

,Season,Squad,Comp,LgRank,MP,W,D,L,GF,GA,GD,Pts,Top Team Scorer,Goalkeeper,Notes
0,2022,Brazil,1. FIFA World Cup,GR,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,2018,Brazil,1. FIFA World Cup,QF,5,3,1,1,8,3,5,10,"Philippe Coutinho, Neymar - 2",Alisson,NaN
2,2014,Brazil,1. FIFA World Cup,4th,7,3,2,2,11,14,-3,11,Neymar - 4,Júlio César,NaN
3,2010,Brazil,1. FIFA World Cup,QF,5,3,1,1,9,4,5,10,Luís Fabiano - 3,Júlio César,NaN
4,2006,Brazil,1. FIFA World Cup,QF,5,4,0,1,10,2,8,12,Ronaldo - 3,Dida,NaN
5,2002,Brazil,1. FIFA World Cup,1st,7,7,0,0,18,4,14,21,Ronaldo - 8,Marcos,NaN
6,1998,Brazil,1. FIFA World Cup,2nd,7,4,1,2,14,10,4,13,Ronaldo - 4,Cláudio Taffarel,NaN
7,1994,Brazil,1. FIFA World Cup,1st,7,5,2,0,11,3,8,17,Romário - 5,Cláudio Taffarel,NaN
8,1990,Brazil,1. FIFA World Cup,R16,4,3,0,1,4,2,2,6,"Careca, Müller - 2",Cláudio Taffarel,NaN
9,1986,Brazil,1. FIFA World Cup,QF,5,4,1,0,10,1,9,9,Careca - 5,Carlos Roberto Gallo,NaN
